# 1.模型构造

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)   # （批量大小，输入维度）
net(X)

tensor([[ 0.0960, -0.0941, -0.2159, -0.0253,  0.0629, -0.1646,  0.0809, -0.4261,
          0.0090,  0.1062],
        [ 0.0244, -0.1645, -0.1614, -0.0567,  0.0813,  0.0560,  0.0514, -0.3828,
          0.0661,  0.1264]], grad_fn=<AddmmBackward0>)

## 自定义块

In [5]:
class MLP(nn.Module):   # 任何一个层或者神经网络都是module的子类
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用`MLP`的父类`Block`的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数`params`（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的正向传播，即如何根据输入`X`返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

## 实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [6]:
net = MLP()
net(X)

tensor([[-0.1156,  0.2826,  0.2846,  0.1005, -0.0904, -0.0043, -0.0321, -0.1694,
          0.0665, -0.0353],
        [-0.1126,  0.1989,  0.3159,  0.0262, -0.1038,  0.0107,  0.1228, -0.1575,
          0.1497, -0.1131]], grad_fn=<AddmmBackward0>)

## 顺序块

In [7]:
class MySequential(nn.Module):   # Sequential复现
    def __init__(self, *args):   # *args表示任何多个无名参数,它是一个tuple;**kwargs表示关键字参数,它是一个dict
        super().__init__()
        for block in args:
            # 这里，`block`是`Module`子类的一个实例。我们把它保存在'Module'类的成员变量
            # `_modules` 中。`block`的类型是OrderedDict。
            self._modules[block] = block   # 以单个下划线开头的变量或方法仅供内部使用（约定，非强制）

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [8]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0383,  0.3867,  0.1425,  0.0987, -0.0265,  0.2605,  0.0040, -0.0102,
         -0.0539, -0.1452],
        [ 0.0016,  0.2378,  0.0468,  0.0882,  0.0200,  0.2684,  0.0100, -0.0576,
         -0.0368, -0.1369]], grad_fn=<AddmmBackward0>)

## 在正向传播函数中执代码

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变。
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及`relu`和`dot`函数。
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数。
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [10]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1182, grad_fn=<SumBackward0>)

## 混合搭配各种组合块的方法

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.2387, grad_fn=<SumBackward0>)

# 2.参数管理

## 具有单隐藏层的多层感知机

In [12]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))   # sequential可以看作是一个torch的list
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.2072],
        [0.2336]], grad_fn=<AddmmBackward0>)

## 访问参数

In [13]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.1607, -0.1732, -0.0598,  0.0815, -0.0589, -0.0444,  0.1679,  0.2330]])), ('bias', tensor([0.2615]))])


## 目标参数

In [14]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2615], requires_grad=True)
tensor([0.2615])


In [15]:
net[2].weight.grad == None

True

## 一次性访问所有参数

函数在调用多个参数时，在列表、元组、集合、字典及其他可迭代对象作为实参，并在前面加 *，解释器将自动进行解包然后传递给多个单变量参数（参数个数要对应相等）

In [16]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [17]:
print([(name, param.shape) for name, param in net[0].named_parameters()])
print([(name, param.shape) for name, param in net.named_parameters()])

[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]
[('0.weight', torch.Size([8, 4])), ('0.bias', torch.Size([8])), ('2.weight', torch.Size([1, 8])), ('2.bias', torch.Size([1]))]


In [18]:
net.state_dict()['2.bias'].data

tensor([0.2615])

## 从嵌套块收集参数

In [19]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.1829],
        [0.1831]], grad_fn=<AddmmBackward0>)

## 查看构建好的网络结构

In [20]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [21]:
rgnet[0][1][0].bias.data

tensor([ 0.4338,  0.3702,  0.2287,  0.4167, -0.2906, -0.4889,  0.2274,  0.4320])

## 内置初始化

In [22]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0010,  0.0141,  0.0107,  0.0067]), tensor(0.))

In [23]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

## 对某些应用不同的初始化方法

In [24]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.3176, -0.1469, -0.3745, -0.1806])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


## 自定义初始化

In [25]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-5.2159, -9.5534, -9.8786, -0.0000],
        [-8.1691, -0.0000,  0.0000,  0.0000]], grad_fn=<SliceBackward0>)

In [27]:
# 直接替换
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -7.5534, -7.8786,  2.0000])

## 参数绑定

In [28]:
# 我们需要给共享层一个名称，以便可以引用它的参数。
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值。
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# 3.自定义层
## 构造一个没有任何参数的自定义层

In [29]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()   # Python3自动添加，不写也行

    def forward(self, X):
        return X - X.mean()

In [30]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

## 将层作为组件合并到构建更复杂的模型中

In [31]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [32]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-1.1176e-08, grad_fn=<MeanBackward0>)

## 带参数的图层

In [33]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [34]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.6918,  0.6160, -1.1473],
        [-0.0599, -0.9290, -0.7135],
        [ 0.0821, -1.0178,  0.8371],
        [ 1.8576,  1.6540, -0.4956],
        [-0.5799,  0.0600,  0.1532]], requires_grad=True)

## 使用自定义层直接执行正向传播计算

In [35]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.6914, 0.0000]])

## 使用自定义层构建模型

In [36]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.0000],
        [2.3324]])

# 4.读写文件
## 加载和保存张量

In [37]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [38]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

## 存储一个张量列表，然后把它们读回内存

In [39]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

## 写入或读取从字符串映射到张量的字典

In [40]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 加载和保存模型参数

In [41]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

## 将模型的参数存储为一个叫做“mlp.params”的文件

In [42]:
torch.save(net.state_dict(), 'mlp.params')

## 实例化了原始多层感知机模型的一个备份。直接读取文件中存储的参数

In [43]:
clone = MLP()  # 先生成网络，再导入其中的参数
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [44]:
# 验证模型参数是否一致
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])